# Resistivity for BO-1X

In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib
import matplotlib.pyplot as plt
import math
from matplotlib import gridspec
import lasio as la
import scipy as sc
import bruges as br
#import mlo_module as mlo
#from my_plot import set_size
width = 433.62
import segyio
from shutil import copyfile
from scipy import interpolate
from scipy.interpolate import make_interp_spline
from IPython.display import SVG # /!\ note the 'SVG' function also in svgutils.compose
import welly as wl
import urllib
import seaborn as sns

from welly import Project 
from welly import Well

%matplotlib inline

path_data ='//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/'
path_las = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/well logs/'
path_wvlt = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/wavelets/'
fig_path = 'M:/LOCRETA_WORKPACKAGE_D/Valdemar_sprint_well_data_Mads_L/MLO documents/Figures/'
path_hrs = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/HRS project/Locreta_proj_Mads_localtest.prj/seismic.dir/'
path_wells =  '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/well logs/locreta_ml/all_wells/'
path_project = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/well logs/locreta_ml/'

In [ ]:
path_bo1x = '//netapp/Petrel/PetrelProjects/UTM33/Employee/PSpro/DATA/Mads/DATA/well logs/locreta_ml/all_wells/wells_res_invest/'

In [ ]:
zones = pd.read_excel(path_project+'Zones-wells-LOCRETA.xlsx')
zones = zones.drop(index=0)
zones['MD'] = zones['MD']*0.3048 #ft to m

In [ ]:
#bo1x = la.read(path_bo1x + 'BO1X_ILD.las')

In [ ]:
#bo1x.to_csv(path_bo1x + 'BO1X_ILD.csv')

In [ ]:
test = pd.read_csv(path_bo1x + 'BO1X_ILD.csv')
test = test[1:].astype(float)
test['Depth'] = test['MD']*0.3048
test['UWI'] = 'BO-1X'
well_list = ['BO-1X']

In [ ]:
df = pd.DataFrame(columns=test.columns.to_list())

In [ ]:
for i in range(len(well_list)):
    print(well_list[i])
    df = df.append(test.loc[(test['UWI'] == well_list[i]) &  (test['Depth'] > zones['MD'][zones.loc[(zones['WellName'] == well_list[i])].index[0]]) & (test['Depth'] < zones['MD'][zones.loc[(zones['WellName'] == well_list[i])].index[-1]])])
    #df = df.append(test.loc[(test['UWI'] == well_list[i]) &  (test['Depth'] > zones['MD'][zones.loc[(zones['WellName'] == well_list[i])].index[0]]) & (test['Depth'] < zones.loc[(zones['WellName'] == well_list[i]) & (zones['ZONE_NAME']=='Valhall')]['MD'].values[0])])
    
    #df = df.append(test.loc[(test['UWI'] == well_list[i]) & (test['Depth'] < zones.loc[(zones['wellName'] == well_list[i]) & (zones['ZONE_NAME']=='Valhall')]['MD'].values[0])])
    #print(zones['MD'][zones.loc[(zones['wellName'] == well_list[i])].index[0]])

df = df.drop_duplicates()   
df = df.replace([np.inf,-999.2500, -9999.00000], np.nan)
df.reset_index(drop=True,inplace=True)

In [ ]:
df

In [ ]:
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
df['Zone'] = np.nan

In [ ]:
test.loc[test['UWI']=='BO-1X']

In [ ]:
for i in range(len(well_list)):
    print(well_list[i])
    for k in range(len(zones.loc[zones['WellName']==well_list[i]])):
        dummy = zones.loc[zones['WellName']==well_list[i]].copy()
        dummy = dummy.reset_index(drop=True)
        df['Zone'][find_nearest(df.loc[df['UWI'] == well_list[i]]['Depth'],dummy['MD'][k]) + df.loc[df['UWI'] == well_list[i]]['Depth'].index[0]] = dummy['ZONE_NAME'][k]

In [ ]:
df['Zone'] = df['Zone'].fillna(method='ffill')

In [ ]:
df['Zone'].unique()

In [ ]:
sns.scatterplot(data=df, x= 'Depth',y='RESISTIVITY_C', hue='Zone')

In [ ]:
df['ILD_log10'] = np.log1p(df['RESISTIVITY_C'])
df['DEPTH'] = df['Depth']
df = df.drop(index = df.loc[df['Zone']=='Valhall'].index.values)

In [ ]:
df.to_csv(path_project+'BO-1X_ILD.csv',index=False)